# --- Day 10: Pipe Maze ---
You use the hang glider to ride the hot air from Desert Island all the way up to the floating metal island. This island is surprisingly cold and there definitely aren't any thermals to glide on, so you leave your hang glider behind.

You wander around for a while, but you don't find any people or animals. However, you do occasionally find signposts labeled "Hot Springs" pointing in a seemingly consistent direction; maybe you can find someone at the hot springs and ask them where the desert-machine parts are made.

The landscape here is alien; even the flowers and trees are made of metal. As you stop to admire some metal grass, you notice something metallic scurry away in your peripheral vision and jump into a big pipe! It didn't look like any animal you've ever seen; if you want a better look, you'll need to get ahead of it.

Scanning the area, you discover that the entire field you're standing on is densely packed with pipes; it was hard to tell at first because they're the same metallic silver color as the "ground". You make a quick sketch of all of the surface pipes you can see (your puzzle input).

The pipes are arranged in a two-dimensional grid of tiles:

- | is a vertical pipe connecting north and south.
- '-' is a horizontal pipe connecting east and west.
- L is a 90-degree bend connecting north and east.
- J is a 90-degree bend connecting north and west.
- 7 is a 90-degree bend connecting south and west.
- F is a 90-degree bend connecting south and east.
- . is ground; there is no pipe in this tile.
- S is the starting position of the animal; there is a pipe on this tile, but your sketch doesn't show what shape the pipe has.

Based on the acoustics of the animal's scurrying, you're confident the pipe that contains the animal is one large, continuous loop.

For example, here is a square loop of pipe:

`.....`

`.F-7.`

`.|.|.`

`.L-J.`

`.....`

If the animal had entered this loop in the northwest corner, the sketch would instead look like this:

`.....`

`.S-7.`

`.|.|.`

`.L-J.`

`.....`

In the above diagram, the S tile is still a 90-degree F bend: you can tell because of how the adjacent pipes connect to it.

Unfortunately, there are also many pipes that aren't connected to the loop! This sketch shows the same loop as above:

`-L|F7`

`7S-7|`

`L|7||`

`-L-J|`

`L|-JF`

In the above diagram, you can still figure out which pipes form the main loop: they're the ones connected to S, pipes those pipes connect to, pipes those pipes connect to, and so on. Every pipe in the main loop connects to its two neighbors (including S, which will have exactly two pipes connecting to it, and which is assumed to connect back to those two pipes).

Here is a sketch that contains a slightly more complex main loop:

`..F7.`

`.FJ|.`

`SJ.L7`

`|F--J`

`LJ...`

Here's the same example sketch with the extra, non-main-loop pipe tiles also shown:

`7-F7-`

`.FJ|7`

`SJLL7`

`|F--J`

`LJ.LJ`

If you want to get out ahead of the animal, you should find the tile in the loop that is farthest from the starting position. Because the animal is in the pipe, it doesn't make sense to measure this by direct distance. Instead, you need to find the tile that would take the longest number of steps along the loop to reach from the starting point - regardless of which way around the loop the animal went.

In the first example with the square loop:

`.....`

`.S-7.`

`.|.|.`

`.L-J.`

`.....`

You can count the distance each tile in the loop is from the starting point like this:

`.....`

`.012.`

`.1.3.`

`.234.`

`.....`

In this example, the farthest point from the start is 4 steps away.

Here's the more complex loop again:

`..F7.`

`.FJ|.`

`SJ.L7`


`|F--J`

`LJ...`

Here are the distances for each tile on that loop:

`..45.`

`.236.`

`01.78`

`14567`

`23...`

Find the single giant loop starting at S. How many steps along the loop does it take to get from the starting position to the point farthest from the starting position?

In [2]:
def getPipes():
    with open("pipes.txt") as file:
        return file.read()

In [3]:
#Formatting
pipes = getPipes()
pipes = pipes.split("\n")
pipes = [list(x) for x in pipes]

#Find 'S' as the starting location:
for i, row in enumerate(pipes):
    for j, char in enumerate(row):
        if char == "S":
            currentCoords = [i, j] #Keeps track of current coordinates
            break

#Find the next move based on the location of the start ("S")
#Check for valid path up (x coordinate - 1)
if pipes[currentCoords[0] - 1][currentCoords[1]] in ['|', 'F', '7']:
    currentCoords[0] -= 1
    previousDirection = "Up"
#Check for valid path down (x coordinate - 1) if there is no valid path up
elif pipes[currentCoords[0] + 1][currentCoords[1]] in ['|', 'L', 'J']:
    currentCoords[0] += 1
    previousDirection = "Down"
#The start has to be connected by exactly two sides, so if it's not up or down, it has to be left and right
else:
    pipes[1] -= 1
    previousDirection = "Left"
    

#Keep track of the number of steps taken
numSteps = 1

#Loop through until we get back to "S"
while pipes[currentCoords[0]][currentCoords[1]] != "S":
    
    #Keep track of the current character
    currentChar = pipes[currentCoords[0]][currentCoords[1]]
    
    if currentChar == "|":
        #If the previous direction was up, that means it came from the bottom, so we need to move up
        if previousDirection == "Up":
            currentCoords[0] -= 1
            previousDirection = "Up"
        #This means if came from above, so we have to move down
        else:
            currentCoords[0] += 1
            previousDirection = "Down"
            
    elif currentChar == "-":
        #If we came from the left, that means we continue to move left
        if previousDirection == "Left":
            currentCoords[1] -= 1
            previousDirection = "Left"
        #This means if came from the right, so we have to keep moving to the right
        else:
            currentCoords[1] += 1
            previousDirection = "Right"
            
    elif currentChar == "L":
        #If coming from the left, move up
        if previousDirection == "Left":
            currentCoords[0] -= 1
            previousDirection = "Up"
        #If coming from up, move to the right
        else:
            currentCoords[1] += 1
            previousDirection = "Right"
            
    elif currentChar == "J":
        #If coming from right, move up
        if previousDirection == "Right":
            currentCoords[0] -= 1
            previousDirection = "Up"
        #If coming from up move to the left
        else:
            currentCoords[1] -= 1
            previousDirection = "Left"
            
    elif currentChar == "7":
        #If coming from the bottom, move left
        if previousDirection == "Up":
            currentCoords[1] -= 1
            previousDirection = "Left"
        #If coming from the right, move down
        else:
            currentCoords[0] += 1
            previousDirection = "Down"
            
    elif currentChar == "F":
        #If coming from the bottom, move right
        if previousDirection == "Up":
            currentCoords[1] += 1
            previousDirection = "Right"
        #If coming from right, move down
        else:
            currentCoords[0] += 1
            previousDirection = "Down"
    
    #Incriment numSteps by 1
    numSteps += 1
    
print(f"Number of steps to the farthest spot from the animal: {numSteps//2}")

Number of steps to the farthest spot from the animal: 6867


# --- Part Two ---
You quickly reach the farthest point of the loop, but the animal never emerges. Maybe its nest is within the area enclosed by the loop?

To determine whether it's even worth taking the time to search for such a nest, you should calculate how many tiles are contained within the loop. For example:

`...........`

`.S-------7.`

`.|F-----7|.`

`.||.....||.`

`.||.....||.`

`.|L-7.F-J|.`

`.|..|.|..|.`

`.L--J.L--J.`

`...........`

The above loop encloses merely four tiles - the two pairs of . in the southwest and southeast (marked I below). The middle . tiles (marked O below) are not in the loop. Here is the same loop again with those regions marked:

`...........`

`.S-------7.`

`.|F-----7|.`

`.||OOOOO||.`

`.||OOOOO||.`

`.|L-7OF-J|.`

`.|II|O|II|.`

`.L--JOL--J.`

`.....O.....`

In fact, there doesn't even need to be a full tile path to the outside for tiles to count as outside the loop - squeezing between pipes is also allowed! Here, I is still within the loop and O is still outside the loop:

`..........`

`.S------7.`

`.|F----7|.`

`.||OOOO||.`

`.||OOOO||.`

`.|L-7F-J|.`

`.|II||II|.`

`.L--JL--J.`

`..........`

In both of the above examples, 4 tiles are enclosed by the loop.

Here's a larger example:

`.F----7F7F7F7F-7....`

`.|F--7||||||||FJ....`

`.||.FJ||||||||L7....`

`FJL7L7LJLJ||LJ.L-7..`

`L--J.L7...LJS7F-7L7.`

`....F-J..F7FJ|L7L7L7`

`....L7.F7||L7|.L7L7|`

`.....|FJLJ|FJ|F7|.LJ`

`....FJL-7.||.||||...`

`....L---J.LJ.LJLJ...`

The above sketch has many random bits of ground, some of which are in the loop (I) and some of which are outside it (O):

`OF----7F7F7F7F-7OOOO`

`O|F--7||||||||FJOOOO`

`O||OFJ||||||||L7OOOO`

`FJL7L7LJLJ||LJIL-7OO`

`L--JOL7IIILJS7F-7L7O`

`OOOOF-JIIF7FJ|L7L7L7`

`OOOOL7IF7||L7|IL7L7|`

`OOOOO|FJLJ|FJ|F7|OLJ`

`OOOOFJL-7O||O||||OOO`

`OOOOL---JOLJOLJLJOOO`

In this larger example, 8 tiles are enclosed by the loop.

Any tile that isn't part of the main loop can count as being enclosed by the loop. Here's another example with many bits of junk pipe lying around that aren't connected to the main loop at all:

`FF7FSF7F7F7F7F7F---7`

`L|LJ||||||||||||F--J`

`FL-7LJLJ||||||LJL-77`

`F--JF--7||LJLJ7F7FJ-`

`L---JF-JLJ.||-FJLJJ7`

`|F|F-JF---7F7-L7L|7|`

`|FFJF7L7F-JF7|JL---7`

`7-L-JL7||F7|L7F-7F7|`

`L.L7LFJ|||||FJL7||LJ`

`L7JLJL-JLJLJL--JLJ.L`

Here are just the tiles that are enclosed by the loop marked with I:

`FF7FSF7F7F7F7F7F---7`

`L|LJ||||||||||||F--J`

`FL-7LJLJ||||||LJL-77`

`F--JF--7||LJLJIF7FJ-`

`L---JF-JLJIIIIFJLJJ7`

`|F|F-JF---7IIIL7L|7|`

`|FFJF7L7F-JF7IIL---7`

`7-L-JL7||F7|L7F-7F7|`

`L.L7LFJ|||||FJL7||LJ`

`L7JLJL-JLJLJL--JLJ.L`

In this last example, 10 tiles are enclosed by the loop.

Figure out whether you have time to search for the nest by calculating the area within the loop. How many tiles are enclosed by the loop?

For this part, I used some help from the AOC subreddit, I had a very similar idea to this method that wouldn't have worked due to a very small issue that this method fixes
It can be found on this reddit thread:
https://www.reddit.com/r/adventofcode/comments/18f1sgh/2023_day_10_part_2_advise_on_part_2/

In [6]:
def spacePipes(pipes):
    """This function adds empty space between the pipes to make it so that we can recurse between pipes that 
    are not connected"""
    #Put spaces between characters so we can recurse between pipes
    #Puts filler spaces between rows
    spacedPipes = []
    for i, row in enumerate(pipes):
        spacedPipes.append([])

        #Calculate the length of the row to avoid extra filler spaces
        if len(row)*2 % 2 == 0:
            rowLength = (len(row) * 2) - 1
        else:
            rowLength = len(row) * 2

        for j in range(rowLength):
            if j % 2 == 0:
                spacedPipes[-1].append(pipes[i][j//2])
            else:
                if pipes[i][j//2] in ["-", "F", "L"] and pipes[i][(j//2) + 1] in ["-", "J", "7"]:
                    spacedPipes[-1].append("-")
                else:
                    spacedPipes[-1].append("#")

    tempSpacedPipes = spacedPipes
    spacedPipes = []
    for i in range(len(tempSpacedPipes) * 2):

        spacedPipes.append([])

        for j in range(len(tempSpacedPipes[i//2])):
            if i % 2 == 0:
                spacedPipes[-1].append(tempSpacedPipes[i//2][j])
            else:
                if tempSpacedPipes[i//2][j] in ["|", "F", "7"] and tempSpacedPipes[(i//2) + 1][j] in ["|", "J", "L"]:
                    spacedPipes[-1].append("|")
                else:
                    spacedPipes[-1].append("#")

    #Gets rid of additional row spacing at the end
    if len(set(spacedPipes[-1])) == 1:
        spacedPipes = spacedPipes[:-1]
    
    return spacedPipes

def fillBoard(pipes, startXCoordinate, startYCoordinate):
    """Fills the pipe map with points marked as outside points"""
    #Mark first point as an outside point
    pipes[startXCoordinate][startYCoordinate] = "O"
    #Keeps track of all coordinates that are outside points
    outsideCoords = [[startXCoordinate, startYCoordinate]]
    previousOutsideCoords = []
    
    #Keep looping until there is no change in the coordinates
    while outsideCoords != previousOutsideCoords:
        previousOutsideCoords = outsideCoords
        #Loop through all coordinates
        for coords in outsideCoords:
            #Checks to make sure the coordinate above the current coordinate is in bounds
            if isValidCoordinate(pipes, coords[0] - 1, coords[1]):
                #If the pipe is marked as either a filler space, or an inside point, mark it as an outside point
                if pipes[coords[0] - 1][coords[1]] in ["#", "I"] and [coords[0] - 1, coords[1]] not in outsideCoords:
                    #Mark as outside point
                    pipes[coords[0] - 1][coords[1]] = "O"
                    #Add coords to outsideCoords
                    outsideCoords.append([coords[0] - 1, coords[1]])

            #Checks to make sure the coordinate below the current coordinate is in bounds
            if isValidCoordinate(pipes, coords[0] + 1, coords[1]):
                #If the pipe is marked as either a filler space, or an inside point, mark it as an outside point
                if pipes[coords[0] + 1][coords[1]] in ["#", "I"] and [coords[0] + 1, coords[1]] not in outsideCoords:
                    #Mark as outside point
                    pipes[coords[0] + 1][coords[1]] = "O"
                    #Add coords to outsideCoords
                    outsideCoords.append([coords[0] + 1, coords[1]])

            #Checks to make sure the coordinate to the left of the current coordinate is in bounds
            if isValidCoordinate(pipes, coords[0], coords[1] - 1):
                #If the pipe is marked as either a filler space, or an inside point, mark it as an outside point
                if pipes[coords[0]][coords[1] - 1] in ["#", "I"]  and [coords[0], coords[1] - 1] not in outsideCoords:
                    #Mark as outside point
                    pipes[coords[0]][coords[1] - 1] = "O"
                    #Add coords to outsideCoords
                    outsideCoords.append([coords[0], coords[1] - 1])

            #Checks to make sure the coordinate to the right of the current coordinate is in bounds
            if isValidCoordinate(pipes, coords[0], coords[1] + 1):
                #If the pipe is marked as either a filler space, or an inside point, mark it as an outside point
                if pipes[coords[0]][coords[1] + 1] in ["#", "I"] and [coords[0], coords[1] + 1] not in outsideCoords:
                    #Mark as outside point
                    pipes[coords[0]][coords[1] + 1] = "O"
                    #Add coords to outsideCoords
                    outsideCoords.append([coords[0], coords[1] + 1])
            
    return pipes

def isValidCoordinate(pipes, xCoordinate, yCoordinate):
    """Returns False if x or y coordinate is out of bounds otherwise return True"""
    if xCoordinate < 0 or xCoordinate > len(pipes)-1:
        return False
    if yCoordinate < 0 or yCoordinate > len(pipes[0])-1:
        return False
    return True

def printPipes(pipes):
    """Prints pipes, pretty much just used for testing purposes"""
    for i in range(len(pipes)):
        for j in range(len(pipes[i])):
            print(pipes[i][j], end="")
        print()

#Formatting
pipes = getPipes()
pipes = pipes.split("\n")
pipes = [list(x) for x in pipes]

#Find 'S' as the starting location:
for i, row in enumerate(pipes):
    for j, char in enumerate(row):
        if char == "S":
            sCoordinate = [i, j] #Keep track of the start Coordinate
            currentCoords = [i, j] #Keeps track of current coordinates
            break

#Find the next move based on the location of the start ("S")
#Check for valid path up (x coordinate - 1)
if pipes[currentCoords[0] - 1][currentCoords[1]] in ['|', 'F', '7']:
    currentCoords[0] -= 1
    previousDirection = "Up"
#Check for valid path down (x coordinate - 1) if there is no valid path up
elif pipes[currentCoords[0] + 1][currentCoords[1]] in ['|', 'L', 'J']:
    currentCoords[0] += 1
    previousDirection = "Down"
#The start has to be connected by exactly two sides, so if it's not up or down, it has to be left and right
else:
    pipes[1] -= 1
    previousDirection = "Left"
    
pathCoords = [[currentCoords[0], currentCoords[1]]]

#Loop through until we get back to "S"
while pipes[currentCoords[0]][currentCoords[1]] != "S":
    
    #Keep track of the current character
    currentChar = pipes[currentCoords[0]][currentCoords[1]]
    
    if currentChar == "|":
        #If the previous direction was up, that means it came from the bottom, so we need to move up
        if previousDirection == "Up":
            currentCoords[0] -= 1
            previousDirection = "Up"
        #This means if came from above, so we have to move down
        else:
            currentCoords[0] += 1
            previousDirection = "Down"
            
    elif currentChar == "-":
        #If we came from the left, that means we continue to move left
        if previousDirection == "Left":
            currentCoords[1] -= 1
            previousDirection = "Left"
        #This means if came from the right, so we have to keep moving to the right
        else:
            currentCoords[1] += 1
            previousDirection = "Right"
            
    elif currentChar == "L":
        #If coming from the left, move up
        if previousDirection == "Left":
            currentCoords[0] -= 1
            previousDirection = "Up"
        #If coming from up, move to the right
        else:
            currentCoords[1] += 1
            previousDirection = "Right"
            
    elif currentChar == "J":
        #If coming from right, move up
        if previousDirection == "Right":
            currentCoords[0] -= 1
            previousDirection = "Up"
        #If coming from up move to the left
        else:
            currentCoords[1] -= 1
            previousDirection = "Left"
            
    elif currentChar == "7":
        #If coming from the bottom, move left
        if previousDirection == "Up":
            currentCoords[1] -= 1
            previousDirection = "Left"
        #If coming from the right, move down
        else:
            currentCoords[0] += 1
            previousDirection = "Down"
            
    elif currentChar == "F":
        #If coming from the bottom, move right
        if previousDirection == "Up":
            currentCoords[1] += 1
            previousDirection = "Right"
        #If coming from right, move down
        else:
            currentCoords[0] += 1
            previousDirection = "Down"
    
    pathCoords.append([currentCoords[0], currentCoords[1]])
        
#Replace non path characters with "I"
for i, row in enumerate(pipes):
    for j in range(len(row)):
        if [i,j] not in pathCoords:
            pipes[i][j] = "I"
            
#Replace S with correct character
#This assumes S has to be not a '|' or a '-'
connectedBottom = pipes[sCoordinate[0] + 1][sCoordinate[1]] in ['|', 'L', 'J']
connectedTop = pipes[sCoordinate[0] - 1][sCoordinate[1]] in ['|', 'F', '7']
connectedRight = pipes[sCoordinate[0]][sCoordinate[1] + 1] in ['-', 'J', '7']
connectedLeft = pipes[sCoordinate[0]][sCoordinate[1] - 1] in ['-', 'F', 'L']
#Checks for F
if connectedRight and connectedBottom:
    pipes[sCoordinate[0]][sCoordinate[1]] = "F"
#Checks for L
if connectedRight and connectedTop:
    pipes[sCoordinate[0]][sCoordinate[1]] = "L"
#Checks for 7
if connectedLeft and connectedBottom:
    pipes[sCoordinate[0]][sCoordinate[1]] = "7"
#Checks for J
if connectedLeft and connectedTop:
    pipes[sCoordinate[0]][sCoordinate[1]] = "J"

    
#Put empty spaces between every row and column
spacedPipes = spacePipes(pipes)
#Fills the map with outside points
spacedPipes = fillBoard(spacedPipes, 0, 0)

#Number of points inside the the loop
numInsidePoints = 0
for row in spacedPipes:
    numInsidePoints += row.count("I")

print(f"Number of points inside the loop: {numInsidePoints}")

Number of points inside the loop: 595


# Notes:

In [8]:
pipes = """..........
.S------7.
.|F----7|.
.||....||.
.||....||.
.|L-7F-J|.
.|..||..|.
.L--JL--J.
.........."""
pipes = pipes.split("\n")
pipes = [list(x) for x in pipes]

print("Turns this:")
#Print board
for i in range(len(pipes)):
    for j in range(len(pipes[i])):
        print(pipes[i][j], end="")
    print()

spacedPipes = spacePipes(pipes)

print("\n\nInto this:")
#Print board
for i in range(len(spacedPipes)):
    for j in range(len(spacedPipes[i])):
        print(spacedPipes[i][j], end="")
    print()

Turns this:
..........
.S------7.
.|F----7|.
.||....||.
.||....||.
.|L-7F-J|.
.|..||..|.
.L--JL--J.
..........


Into this:
.#.#.#.#.#.#.#.#.#.
###################
.#S#------------7#.
################|##
.#|#F---------7#|#.
##|#|#########|#|##
.#|#|#.#.#.#.#|#|#.
##|#|#########|#|##
.#|#|#.#.#.#.#|#|#.
##|#|#########|#|##
.#|#L---7#F---J#|#.
##|#####|#|#####|##
.#|#.#.#|#|#.#.#|#.
##|#####|#|#####|##
.#L-----J#L-----J#.
###################
.#.#.#.#.#.#.#.#.#.


In [ ]:
"""This flood fill method works great, except for that on too big of an input (like my input) it recurses too deep, and
the kernel dies. It's unclear as to why the kernel dies rather than us getting a recursion depth error, but either way, it
doesn't work"""

def traverseBoard(pipes, xCoordinate, yCoordinate, outsideCoords=[]):
    #Mark current point as an outside point
    outsideCoords.append([xCoordinate, yCoordinate])
    
    #Checks to make sure the coordinate above the current coordinate is in bounds
    if isValidCoordinate(pipes, xCoordinate - 1, yCoordinate):
        #If the pipe is marked as either a filler space, or an inside point, call this function on that point
        if pipes[xCoordinate - 1][yCoordinate] in ["#", "I"] and [xCoordinate - 1, yCoordinate] not in outsideCoords:
            outsideCoords = traverseBoard(pipes, xCoordinate - 1, yCoordinate, outsideCoords)
    
    #Checks to make sure the coordinate below the current coordinate is in bounds
    if isValidCoordinate(pipes, xCoordinate + 1, yCoordinate):
        #If the pipe is marked as either a filler space, or an inside point, call this function on that point
        if pipes[xCoordinate + 1][yCoordinate] in ["#", "I"] and [xCoordinate + 1, yCoordinate] not in outsideCoords:
            outsideCoords = traverseBoard(pipes, xCoordinate + 1, yCoordinate, outsideCoords)
    
    #Checks to make sure the coordinate to the left of the current coordinate is in bounds
    if isValidCoordinate(pipes, xCoordinate, yCoordinate - 1):
        #If the pipe is marked as either a filler space, or an inside point, call this function on that point
        if pipes[xCoordinate][yCoordinate - 1] in ["#", "I"]  and [xCoordinate, yCoordinate - 1] not in outsideCoords:
            outsideCoords = traverseBoard(pipes, xCoordinate, yCoordinate - 1, outsideCoords)
    
    #Checks to make sure the coordinate to the right of the current coordinate is in bounds
    if isValidCoordinate(pipes, xCoordinate, yCoordinate + 1):
        #If the pipe is marked as either a filler space, or an inside point, call this function on that point
        if pipes[xCoordinate][yCoordinate + 1] in ["#", "I"] and [xCoordinate, yCoordinate + 1] not in outsideCoords:
            outsideCoords = traverseBoard(pipes, xCoordinate, yCoordinate + 1, outsideCoords)
            
    return outsideCoords

"""
spacedPipes = spacePipes(pipes)
for coords in traverseBoard(spacedPipes, 0, 0):
    spacedPipes[coords[0]][coords[1]] = "O"
"""

In [ ]:
#TEST INPUTS
"""
..........
.S------7.
.|F----7|.
.||....||.
.||....||.
.|L-7F-J|.
.|..||..|.
.L--JL--J.
..........
"""
"""
......................
..F----7F7F7F7F-7.....
..|F--7||||||||FJ.....
..||.FJ||||||||L7.....
.FJL7L7LJLJ||LJ.L-7...
.L--J.L7...LJS7F-7L7..
.....F-J..F7FJ|L7L7L7.
.....L7.F7||L7|.L7L7|.
......|FJLJ|FJ|F7|.LJ.
.....FJL-7.||.||||....
.....L---J.LJ.LJLJ....
......................
"""
"""
......................
.FF7FSF7F7F7F7F7F---7.
.L|LJ||||||||||||F--J.
.FL-7LJLJ||||||LJL-77.
.F--JF--7||LJLJIF7FJ-.
.L---JF-JLJIIIIFJLJJ7.
.|F|F-JF---7IIIL7L|7|.
.|FFJF7L7F-JF7IIL---7.
.7-L-JL7||F7|L7F-7F7|.
.L.L7LFJ|||||FJL7||LJ.
.L7JLJL-JLJLJL--JLJ.L.
......................
"""